### Dataframe operations

In [3]:
from pyspark.sql import Row

row = Row(name="Alice", age=11)
print(row)
print(row['name'], row['age'])
print(row.name, row.age)

row = Row(name="Alice", age=11, count=1)
print(row.count)
print(row['count'])

Row(name='Alice', age=11)
Alice 11
Alice 11
<built-in method count of Row object at 0x7ff184030258>
1


In [4]:
# Data file at https://www.cse.ust.hk/msbd5003/data/building.csv

df = spark.read.csv('../data/building.csv', header=True, inferSchema=True)

In [5]:
# show the content of the dataframe
df.show()

+----------+-----------+-----------+-----------+------------+
|BuildingID|BuildingMgr|BuildingAge|HVACproduct|     Country|
+----------+-----------+-----------+-----------+------------+
|         1|         M1|         25|     AC1000|         USA|
|         2|         M2|         27|     FN39TG|      France|
|         3|         M3|         28|     JDNS77|      Brazil|
|         4|         M4|         17|     GG1919|     Finland|
|         5|         M5|          3|    ACMAX22|   Hong Kong|
|         6|         M6|          9|     AC1000|   Singapore|
|         7|         M7|         13|     FN39TG|South Africa|
|         8|         M8|         25|     JDNS77|   Australia|
|         9|         M9|         11|     GG1919|      Mexico|
|        10|        M10|         23|    ACMAX22|       China|
|        11|        M11|         14|     AC1000|     Belgium|
|        12|        M12|         26|     FN39TG|     Finland|
|        13|        M13|         25|     JDNS77|Saudi Arabia|
|       

In [6]:
# Print the dataframe schema in a tree format
df.printSchema()

root
 |-- BuildingID: integer (nullable = true)
 |-- BuildingMgr: string (nullable = true)
 |-- BuildingAge: integer (nullable = true)
 |-- HVACproduct: string (nullable = true)
 |-- Country: string (nullable = true)



In [7]:
# Create an RDD from the dataframe
dfrdd = df.rdd
dfrdd.take(3)

[Row(BuildingID=1, BuildingMgr='M1', BuildingAge=25, HVACproduct='AC1000', Country='USA'),
 Row(BuildingID=2, BuildingMgr='M2', BuildingAge=27, HVACproduct='FN39TG', Country='France'),
 Row(BuildingID=3, BuildingMgr='M3', BuildingAge=28, HVACproduct='JDNS77', Country='Brazil')]

In [8]:
# Retrieve specific columns from the dataframe
df.select('BuildingID', 'Country').show()

+----------+------------+
|BuildingID|     Country|
+----------+------------+
|         1|         USA|
|         2|      France|
|         3|      Brazil|
|         4|     Finland|
|         5|   Hong Kong|
|         6|   Singapore|
|         7|South Africa|
|         8|   Australia|
|         9|      Mexico|
|        10|       China|
|        11|     Belgium|
|        12|     Finland|
|        13|Saudi Arabia|
|        14|     Germany|
|        15|      Israel|
|        16|      Turkey|
|        17|       Egypt|
|        18|   Indonesia|
|        19|      Canada|
|        20|   Argentina|
+----------+------------+



In [9]:
from pyspark.sql.functions import *

df.where("Country<'USA'").select('BuildingID', lit('OK')).show()

+----------+---+
|BuildingID| OK|
+----------+---+
|         2| OK|
|         3| OK|
|         4| OK|
|         5| OK|
|         6| OK|
|         7| OK|
|         8| OK|
|         9| OK|
|        10| OK|
|        11| OK|
|        12| OK|
|        13| OK|
|        14| OK|
|        15| OK|
|        16| OK|
|        17| OK|
|        18| OK|
|        19| OK|
|        20| OK|
+----------+---+



In [10]:
# Use GroupBy clause with dataframe 
df.groupBy('HVACProduct').count().show()

+-----------+-----+
|HVACProduct|count|
+-----------+-----+
|    ACMAX22|    4|
|     AC1000|    4|
|     JDNS77|    4|
|     FN39TG|    4|
|     GG1919|    4|
+-----------+-----+



### Rewriting SQL with DataFrame API

In [11]:
# Load data from csv files
# Data files at https://www.cse.ust.hk/msbd5003/data

dfCustomer = spark.read.csv('../data/Customer.csv', header=True, inferSchema=True)
dfProduct = spark.read.csv('../data/Product.csv', header=True, inferSchema=True)
dfDetail = spark.read.csv('../data/SalesOrderDetail.csv', header=True, inferSchema=True)
dfHeader = spark.read.csv('../data/SalesOrderHeader.csv', header=True, inferSchema=True)

In [12]:
# SELECT ProductID, Name, ListPrice 
# FROM Product 
# WHERE Color = 'black'

dfProduct.filter("Color = 'Black'")\
         .select('ProductID', 'Name', 'ListPrice')\
         .show(truncate=False)

+---------+-----------------------------+---------+
|ProductID|Name                         |ListPrice|
+---------+-----------------------------+---------+
|680      |HL Road Frame - Black, 58    |1431.5   |
|708      |Sport-100 Helmet, Black      |34.99    |
|722      |LL Road Frame - Black, 58    |337.22   |
|723      |LL Road Frame - Black, 60    |337.22   |
|724      |LL Road Frame - Black, 62    |337.22   |
|736      |LL Road Frame - Black, 44    |337.22   |
|737      |LL Road Frame - Black, 48    |337.22   |
|738      |LL Road Frame - Black, 52    |337.22   |
|743      |HL Mountain Frame - Black, 42|1349.6   |
|744      |HL Mountain Frame - Black, 44|1349.6   |
|745      |HL Mountain Frame - Black, 48|1349.6   |
|746      |HL Mountain Frame - Black, 46|1349.6   |
|747      |HL Mountain Frame - Black, 38|1349.6   |
|765      |Road-650 Black, 58           |782.99   |
|766      |Road-650 Black, 60           |782.99   |
|767      |Road-650 Black, 62           |782.99   |
|768      |R

In [13]:
dfProduct.where(dfProduct.Color=='Black') \
         .select(dfProduct.ProductID, dfProduct['Name'], (dfProduct['ListPrice'] * 2).alias('Double price')) \
         .show(truncate=False)

+---------+-----------------------------+------------+
|ProductID|Name                         |Double price|
+---------+-----------------------------+------------+
|680      |HL Road Frame - Black, 58    |2863.0      |
|708      |Sport-100 Helmet, Black      |69.98       |
|722      |LL Road Frame - Black, 58    |674.44      |
|723      |LL Road Frame - Black, 60    |674.44      |
|724      |LL Road Frame - Black, 62    |674.44      |
|736      |LL Road Frame - Black, 44    |674.44      |
|737      |LL Road Frame - Black, 48    |674.44      |
|738      |LL Road Frame - Black, 52    |674.44      |
|743      |HL Mountain Frame - Black, 42|2699.2      |
|744      |HL Mountain Frame - Black, 44|2699.2      |
|745      |HL Mountain Frame - Black, 48|2699.2      |
|746      |HL Mountain Frame - Black, 46|2699.2      |
|747      |HL Mountain Frame - Black, 38|2699.2      |
|765      |Road-650 Black, 58           |1565.98     |
|766      |Road-650 Black, 60           |1565.98     |
|767      

In [14]:
dfProduct.where(dfProduct.ListPrice * 2 > 100) \
         .select(dfProduct.ProductID, dfProduct['Name'], dfProduct.ListPrice * 2) \
         .show(truncate=False)

+---------+-------------------------+---------------+
|ProductID|Name                     |(ListPrice * 2)|
+---------+-------------------------+---------------+
|680      |HL Road Frame - Black, 58|2863.0         |
|706      |HL Road Frame - Red, 58  |2863.0         |
|717      |HL Road Frame - Red, 62  |2863.0         |
|718      |HL Road Frame - Red, 44  |2863.0         |
|719      |HL Road Frame - Red, 48  |2863.0         |
|720      |HL Road Frame - Red, 52  |2863.0         |
|721      |HL Road Frame - Red, 56  |2863.0         |
|722      |LL Road Frame - Black, 58|674.44         |
|723      |LL Road Frame - Black, 60|674.44         |
|724      |LL Road Frame - Black, 62|674.44         |
|725      |LL Road Frame - Red, 44  |674.44         |
|726      |LL Road Frame - Red, 48  |674.44         |
|727      |LL Road Frame - Red, 52  |674.44         |
|728      |LL Road Frame - Red, 58  |674.44         |
|729      |LL Road Frame - Red, 60  |674.44         |
|730      |LL Road Frame - R

In [15]:
# SELECT ProductID, Name, ListPrice 
# FROM Product 
# WHERE Color = 'black' 
# ORDER BY ProductID

dfProduct.filter("Color = 'Black'")\
         .select('ProductID', 'Name', 'ListPrice')\
         .orderBy('ListPrice')\
         .show(truncate=False)

+---------+--------------------------+---------+
|ProductID|Name                      |ListPrice|
+---------+--------------------------+---------+
|860      |Half-Finger Gloves, L     |24.49    |
|859      |Half-Finger Gloves, M     |24.49    |
|858      |Half-Finger Gloves, S     |24.49    |
|708      |Sport-100 Helmet, Black   |34.99    |
|862      |Full-Finger Gloves, M     |37.99    |
|861      |Full-Finger Gloves, S     |37.99    |
|863      |Full-Finger Gloves, L     |37.99    |
|841      |Men's Sports Shorts, S    |59.99    |
|849      |Men's Sports Shorts, M    |59.99    |
|851      |Men's Sports Shorts, XL   |59.99    |
|850      |Men's Sports Shorts, L    |59.99    |
|815      |LL Mountain Front Wheel   |60.745   |
|868      |Women's Mountain Shorts, M|69.99    |
|869      |Women's Mountain Shorts, L|69.99    |
|867      |Women's Mountain Shorts, S|69.99    |
|853      |Women's Tights, M         |74.99    |
|854      |Women's Tights, L         |74.99    |
|852      |Women's T

In [16]:
# Find all orders and details on black product,
# return the product SalesOrderID, SalesOrderDetailID, Name, UnitPrice, and OrderQty

# SELECT SalesOrderID, SalesOrderDetailID, Name, UnitPrice, OrderQty 
# FROM SalesLT.SalesOrderDetail, SalesLT.Product
# WHERE SalesOrderDetail.ProductID = Product.ProductID AND Color = 'Black'

# SELECT SalesOrderID, SalesOrderDetailID, Name, UnitPrice, OrderQty 
# FROM SalesLT.SalesOrderDetail
# JOIN SalesLT.Product ON SalesOrderDetail.ProductID = Product.ProductID
# WHERE Color = 'Black'

# Spark SQL supports natural joins

dfDetail.join(dfProduct, 'ProductID') \
        .select('SalesOrderID', 'SalesOrderDetailID', 'Name', 'UnitPrice', 'OrderQty') \
        .filter("Color='Black'") \
        .show()

# If we move the filter to after select, it still works.  Why?

+------------+------------------+--------------------+---------+--------+
|SalesOrderID|SalesOrderDetailID|                Name|UnitPrice|OrderQty|
+------------+------------------+--------------------+---------+--------+
|       71938|            113295|Sport-100 Helmet,...|   20.994|       5|
|       71902|            112988|Sport-100 Helmet,...|   20.994|       4|
|       71797|            111082|Sport-100 Helmet,...|  20.2942|      12|
|       71784|            110795|Sport-100 Helmet,...|  20.2942|      12|
|       71783|            110752|Sport-100 Helmet,...|  20.2942|      11|
|       71782|            110690|Sport-100 Helmet,...|   20.994|       7|
|       71797|            111045|LL Road Frame - B...|  202.332|       3|
|       71783|            110730|LL Road Frame - B...|  202.332|       6|
|       71938|            113297|LL Road Frame - B...|  202.332|       3|
|       71915|            113090|LL Road Frame - B...|  202.332|       2|
|       71815|            111451|LL Ro

In [17]:
# This also works:

d1 = dfDetail.join(dfProduct, 'ProductID') \
             .select('SalesOrderID', 'SalesOrderDetailID', 'Name', 'UnitPrice', 'OrderQty')
d1.show()
d2 = d1.filter("Color = 'Black'")
d2.show()
d2.explain()

+------------+------------------+--------------------+---------+--------+
|SalesOrderID|SalesOrderDetailID|                Name|UnitPrice|OrderQty|
+------------+------------------+--------------------+---------+--------+
|       71938|            113278|Sport-100 Helmet,...|   20.994|       3|
|       71936|            113228|Sport-100 Helmet,...|   20.994|       1|
|       71902|            112980|Sport-100 Helmet,...|   20.994|       2|
|       71797|            111075|Sport-100 Helmet,...|   20.994|       6|
|       71784|            110794|Sport-100 Helmet,...|   20.994|      10|
|       71783|            110751|Sport-100 Helmet,...|   20.994|      10|
|       71782|            110709|Sport-100 Helmet,...|   20.994|       3|
|       71938|            113295|Sport-100 Helmet,...|   20.994|       5|
|       71902|            112988|Sport-100 Helmet,...|   20.994|       4|
|       71797|            111082|Sport-100 Helmet,...|  20.2942|      12|
|       71784|            110795|Sport

In [18]:
# This will report an error:

d1 = dfDetail.join(dfProduct, 'ProductID') \
             .select('SalesOrderID', 'SalesOrderDetailID', 'Name', 'UnitPrice', 'OrderQty')
d1.write.csv('temp.csv', mode = 'overwrite', header = True)
d2 = spark.read.csv('temp.csv', header = True, inferSchema = True)
d2.filter("Color = 'Black'").show()


AnalysisException: cannot resolve '`Color`' given input columns: [Name, OrderQty, SalesOrderDetailID, SalesOrderID, UnitPrice]; line 1 pos 0;
'Filter ('Color = Black)
+- Relation[SalesOrderID#580,SalesOrderDetailID#581,Name#582,UnitPrice#583,OrderQty#584] csv


In [19]:
# Find all orders that include at least one black product, 
# return the product SalesOrderID, Name, UnitPrice, and OrderQty

# SELECT DISTINCT SalesOrderID
# FROM SalesLT.SalesOrderDetail
# JOIN SalesLT.Product ON SalesOrderDetail.ProductID = Product.ProductID
# WHERE Color = 'Black'

dfDetail.join(dfProduct.filter("Color='Black'"), 'ProductID') \
        .select('SalesOrderID') \
        .distinct() \
        .show()

+------------+
|SalesOrderID|
+------------+
|       71902|
|       71832|
|       71915|
|       71831|
|       71898|
|       71935|
|       71938|
|       71845|
|       71783|
|       71815|
|       71936|
|       71863|
|       71780|
|       71782|
|       71899|
|       71784|
|       71797|
+------------+



In [20]:
# How many colors in the products?

# SELECT COUNT(DISTINCT Color)
# FROM SalesLT.Product

dfProduct.select('Color').distinct().count()

# It's 1 more than standard SQL.  In standard SQL, COUNT() does not count NULLs.

10

In [21]:
# Find the total price of each order, 
# return SalesOrderID and total price (column name should be ‘totalprice’)

# SELECT SalesOrderID, SUM(UnitPrice*OrderQty*(1-UnitPriceDiscount)) AS TotalPrice
# FROM SalesLT.SalesOrderDetail
# GROUP BY SalesOrderID

dfDetail.select('*', (dfDetail.UnitPrice * dfDetail.OrderQty
                      * (1 - dfDetail.UnitPriceDiscount)).alias('netprice'))\
        .groupBy('SalesOrderID').sum('netprice') \
        .withColumnRenamed('sum(netprice)', 'TotalPrice')\
        .show()

+------------+------------------+
|SalesOrderID|        TotalPrice|
+------------+------------------+
|       71867|             858.9|
|       71902|59894.209199999976|
|       71832|      28950.678108|
|       71915|1732.8899999999999|
|       71946|            31.584|
|       71895|221.25600000000003|
|       71816|2847.4079999999994|
|       71831|          1712.946|
|       71923|         96.108824|
|       71858|11528.844000000001|
|       71917|            37.758|
|       71897|          10585.05|
|       71885|           524.664|
|       71856|500.30400000000003|
|       71898| 53248.69200000002|
|       71774|           713.796|
|       71796| 47848.02600000001|
|       71935|5533.8689079999995|
|       71938|         74160.228|
|       71845|        34118.5356|
+------------+------------------+
only showing top 20 rows



In [22]:
# Find the total price of each order where the total price > 10000

# SELECT SalesOrderID, SUM(UnitPrice*OrderQty*(1-UnitPriceDiscount)) AS TotalPrice
# FROM SalesLT.SalesOrderDetail
# GROUP BY SalesOrderID
# HAVING SUM(UnitPrice*OrderQty*(1-UnitPriceDiscount)) > 10000

dfDetail.select('*', (dfDetail.UnitPrice * dfDetail. OrderQty
                      * (1 - dfDetail.UnitPriceDiscount)).alias('netprice'))\
        .groupBy('SalesOrderID').sum('netprice') \
        .withColumnRenamed('sum(netprice)', 'TotalPrice')\
        .where('TotalPrice > 10000')\
        .show()

+------------+------------------+
|SalesOrderID|        TotalPrice|
+------------+------------------+
|       71902|59894.209199999976|
|       71832|      28950.678108|
|       71858|11528.844000000001|
|       71897|          10585.05|
|       71898| 53248.69200000002|
|       71796| 47848.02600000001|
|       71938|         74160.228|
|       71845|        34118.5356|
|       71783|      65683.367986|
|       71936| 79589.61602399996|
|       71780|29923.007999999998|
|       71782| 33319.98600000001|
|       71784| 89869.27631400003|
|       71797| 65123.46341800001|
+------------+------------------+



In [23]:
# Find the total price on the black products of each order where the total price > 10000

# SELECT SalesOrderID, SUM(UnitPrice*OrderQty*(1-UnitPriceDiscount)) AS TotalPrice
# FROM SalesLT.SalesOrderDetail, SalesLT.Product
# WHERE SalesLT.SalesOrderDetail.ProductID = SalesLT.Product.ProductID AND Color = 'Black'
# GROUP BY SalesOrderID
# HAVING SUM(UnitPrice*OrderQty*(1-UnitPriceDiscount)) > 10000

dfDetail.select('*', (dfDetail.UnitPrice * dfDetail. OrderQty
                      * (1 - dfDetail.UnitPriceDiscount)).alias('netprice'))\
        .join(dfProduct.where("Color = 'Black'"), 'ProductID') \
        .groupBy('SalesOrderID').sum('netprice') \
        .withColumnRenamed('sum(netprice)', 'TotalPrice')\
        .where('TotalPrice > 10000')\
        .show()

+------------+------------------+
|SalesOrderID|        TotalPrice|
+------------+------------------+
|       71902|26677.883999999995|
|       71832|      16883.748108|
|       71938|         33779.448|
|       71845|         18109.836|
|       71783|15524.117476000003|
|       71936|      44490.290424|
|       71780|         16964.322|
|       71797|      27581.613792|
+------------+------------------+



In [24]:
# For each customer, find the total quantity of black products bought.
# Report CustomerID, FirstName, LastName, and total quantity

# select saleslt.customer.customerid, FirstName, LastName, sum(orderqty)
# from saleslt.customer
# left outer join 
# (
# saleslt.salesorderheader
# join saleslt.salesorderdetail
# on saleslt.salesorderdetail.salesorderid = saleslt.salesorderheader.salesorderid
# join saleslt.product
# on saleslt.product.productid = saleslt.salesorderdetail.productid and color = 'black'
# )
# on saleslt.customer.customerid = saleslt.salesorderheader.customerid
# group by saleslt.customer.customerid, FirstName, LastName
# order by sum(orderqty) desc

d1 = dfDetail.join(dfProduct, 'ProductID')\
             .where('Color = "Black"') \
             .join(dfHeader, 'SalesOrderID')\
             .groupBy('CustomerID').sum('OrderQty')
dfCustomer.join(d1, 'CustomerID', 'left_outer')\
          .select('CustomerID', 'FirstName', 'LastName', 'sum(OrderQty)')\
          .orderBy('sum(OrderQty)', ascending=False)\
          .show()

+----------+------------+------------+-------------+
|CustomerID|   FirstName|    LastName|sum(OrderQty)|
+----------+------------+------------+-------------+
|     30050|     Krishna|Sunkammurali|           89|
|     29796|         Jon|      Grande|           65|
|     29957|       Kevin|         Liu|           62|
|     29929|     Jeffrey|       Kurtz|           46|
|     29546| Christopher|        Beck|           45|
|     29922|      Pamala|        Kotc|           34|
|     30113|        Raja|   Venugopal|           34|
|     29938|       Frank|    Campbell|           29|
|     29736|       Terry|   Eminhizer|           23|
|     29485|   Catherine|        Abel|           10|
|     30019|     Matthew|      Miller|            9|
|     29932|     Rebecca|      Laszlo|            7|
|     29975|      Walter|        Mays|            5|
|     29638|    Rosmarie|     Carroll|            2|
|     30089|Michael John|      Troyer|            1|
|     29531|        Cory|       Booth|        

-------

### Embed SQL queries

You can also run SQL queries over dataframes once you register them as temporary tables within the SparkSession.

In [25]:
# Register the dataframe as a temporary view called HVAC
df.createOrReplaceTempView('HVAC')

In [26]:
spark.sql('SELECT * FROM HVAC WHERE BuildingAge >= 10').show()

+----------+-----------+-----------+-----------+------------+
|BuildingID|BuildingMgr|BuildingAge|HVACproduct|     Country|
+----------+-----------+-----------+-----------+------------+
|         1|         M1|         25|     AC1000|         USA|
|         2|         M2|         27|     FN39TG|      France|
|         3|         M3|         28|     JDNS77|      Brazil|
|         4|         M4|         17|     GG1919|     Finland|
|         7|         M7|         13|     FN39TG|South Africa|
|         8|         M8|         25|     JDNS77|   Australia|
|         9|         M9|         11|     GG1919|      Mexico|
|        10|        M10|         23|    ACMAX22|       China|
|        11|        M11|         14|     AC1000|     Belgium|
|        12|        M12|         26|     FN39TG|     Finland|
|        13|        M13|         25|     JDNS77|Saudi Arabia|
|        14|        M14|         17|     GG1919|     Germany|
|        15|        M15|         19|    ACMAX22|      Israel|
|       

In [27]:
# Can even mix DataFrame API with SQL:
df.where('BuildingAge >= 10').createOrReplaceTempView('OldBuildings')
spark.sql('SELECT HVACproduct, COUNT(*) FROM OldBuildings GROUP BY HVACproduct').show()

+-----------+--------+
|HVACproduct|count(1)|
+-----------+--------+
|    ACMAX22|       3|
|     AC1000|       3|
|     JDNS77|       4|
|     FN39TG|       4|
|     GG1919|       4|
+-----------+--------+



In [28]:
d1 = spark.sql('SELECT * FROM HVAC WHERE BuildingAge >= 10')
d1.groupBy('HVACproduct').count().show()

+-----------+-----+
|HVACproduct|count|
+-----------+-----+
|    ACMAX22|    3|
|     AC1000|    3|
|     JDNS77|    4|
|     FN39TG|    4|
|     GG1919|    4|
+-----------+-----+



In [29]:
# UDF

from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType

slen = udf(lambda s: len(s)+2, IntegerType())
df.select('*', slen(df['Country']).alias('slen')).show()

+----------+-----------+-----------+-----------+------------+----+
|BuildingID|BuildingMgr|BuildingAge|HVACproduct|     Country|slen|
+----------+-----------+-----------+-----------+------------+----+
|         1|         M1|         25|     AC1000|         USA|   5|
|         2|         M2|         27|     FN39TG|      France|   8|
|         3|         M3|         28|     JDNS77|      Brazil|   8|
|         4|         M4|         17|     GG1919|     Finland|   9|
|         5|         M5|          3|    ACMAX22|   Hong Kong|  11|
|         6|         M6|          9|     AC1000|   Singapore|  11|
|         7|         M7|         13|     FN39TG|South Africa|  14|
|         8|         M8|         25|     JDNS77|   Australia|  11|
|         9|         M9|         11|     GG1919|      Mexico|   8|
|        10|        M10|         23|    ACMAX22|       China|   7|
|        11|        M11|         14|     AC1000|     Belgium|   9|
|        12|        M12|         26|     FN39TG|     Finland| 

In [30]:
spark.udf.register('slen', lambda s: len(s), IntegerType())
spark.sql('SELECT *, slen(Country) AS slen FROM HVAC').show()

+----------+-----------+-----------+-----------+------------+----+
|BuildingID|BuildingMgr|BuildingAge|HVACproduct|     Country|slen|
+----------+-----------+-----------+-----------+------------+----+
|         1|         M1|         25|     AC1000|         USA|   3|
|         2|         M2|         27|     FN39TG|      France|   6|
|         3|         M3|         28|     JDNS77|      Brazil|   6|
|         4|         M4|         17|     GG1919|     Finland|   7|
|         5|         M5|          3|    ACMAX22|   Hong Kong|   9|
|         6|         M6|          9|     AC1000|   Singapore|   9|
|         7|         M7|         13|     FN39TG|South Africa|  12|
|         8|         M8|         25|     JDNS77|   Australia|   9|
|         9|         M9|         11|     GG1919|      Mexico|   6|
|        10|        M10|         23|    ACMAX22|       China|   5|
|        11|        M11|         14|     AC1000|     Belgium|   7|
|        12|        M12|         26|     FN39TG|     Finland| 

---
## Flexible Data Model

Sample data file at

https://www.cse.ust.hk/msbd5003/data/products.json

In [31]:
df = spark.read.json('../data/products.json')
df.printSchema()

root
 |-- dimensions: struct (nullable = true)
 |    |-- height: double (nullable = true)
 |    |-- length: double (nullable = true)
 |    |-- width: double (nullable = true)
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- price: double (nullable = true)
 |-- tags: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- warehouseLocation: struct (nullable = true)
 |    |-- latitude: double (nullable = true)
 |    |-- longitude: double (nullable = true)



In [32]:
df.show()

+----------------+---+----------------+-----+-----------+-----------------+
|      dimensions| id|            name|price|       tags|warehouseLocation|
+----------------+---+----------------+-----+-----------+-----------------+
|[9.5, 7.0, 12.0]|  2|An ice sculpture| 12.5|[cold, ice]|   [-78.75, 20.4]|
| [1.0, 3.1, 1.0]|  3|    A blue mouse| 25.5|       null|    [54.4, -32.7]|
+----------------+---+----------------+-----+-----------+-----------------+



In [33]:
# Accessing nested fields

df.select(df['dimensions.height']).show()

+------+
|height|
+------+
|   9.5|
|   1.0|
+------+



In [34]:
df.select('dimensions.height').show()

+------+
|height|
+------+
|   9.5|
|   1.0|
+------+



In [35]:
df.select('dimensions.height')\
  .filter("tags[0] = 'cold' AND warehouseLocation.latitude < 0")\
  .show()

+------+
|height|
+------+
|   9.5|
+------+



In [36]:
df.rdd.take(3)

[Row(dimensions=Row(height=9.5, length=7.0, width=12.0), id=2, name='An ice sculpture', price=12.5, tags=['cold', 'ice'], warehouseLocation=Row(latitude=-78.75, longitude=20.4)),
 Row(dimensions=Row(height=1.0, length=3.1, width=1.0), id=3, name='A blue mouse', price=25.5, tags=None, warehouseLocation=Row(latitude=54.4, longitude=-32.7))]

---
## Converting between RDD and DataFrame

Sample data file at:

https://www.cse.ust.hk/msbd5003/data/people.txt

In [37]:
# Load a text file and convert each line to a Row.
lines = sc.textFile("../data/people.txt")

def parse(l):
    a = l.split(',')
    return (a[0], int(a[1]))

rdd = lines.map(parse)
rdd.collect()

[('Michael', 29), ('Andy', 30), ('Justin', 19)]

In [38]:
# Create the DataFrame from an RDD of tuples, schema is inferred
df = spark.createDataFrame(rdd)
df.printSchema()
df.show()

root
 |-- _1: string (nullable = true)
 |-- _2: long (nullable = true)

+-------+---+
|     _1| _2|
+-------+---+
|Michael| 29|
|   Andy| 30|
| Justin| 19|
+-------+---+



In [39]:
# Create the DataFrame from an RDD of tuples with column names, type is inferred
df = spark.createDataFrame(rdd, ['name', 'age'])
df.printSchema()
df.show()

root
 |-- name: string (nullable = true)
 |-- age: long (nullable = true)

+-------+---+
|   name|age|
+-------+---+
|Michael| 29|
|   Andy| 30|
| Justin| 19|
+-------+---+



In [40]:
# Create the DataFrame from an RDD of Rows, type is given in the Row objects
from pyspark.sql import Row

rdd_rows = rdd.map(lambda p: Row(name = p[0], age = p[1]))
df = spark.createDataFrame(rdd_rows)
df.printSchema()
df.show()

root
 |-- name: string (nullable = true)
 |-- age: long (nullable = true)

+-------+---+
|   name|age|
+-------+---+
|Michael| 29|
|   Andy| 30|
| Justin| 19|
+-------+---+



In [41]:
# Row fields with types incompatible with that of previous rows will be turned into nulls
row1 = Row(name="Alice", age=11)
row2 = Row(name="Bob", age='12')
rdd_rows = sc.parallelize([row1, row2])
df1 = spark.createDataFrame(rdd_rows)
df1.show()

+-----+----+
| name| age|
+-----+----+
|Alice|  11|
|  Bob|null|
+-----+----+



In [42]:
# rdd returns the content as an RDD of Rows
teenagers = df.filter('age >= 13 and age <= 19')

teenNames = teenagers.rdd.map(lambda p: "Name: " + p.name)
teenNames.collect()

['Name: Justin']

### Note:

DataFrames are stored using columnar storage with compression

RDDs are stored using row storage without compression

The RDD view of DataFrame just provides an interface, the Row objects are constructed on the fly and do not necessarily represent the internal storage format of the data

### Closure in DataFrames

In [43]:
data = range(10)
df = spark.createDataFrame(zip(data, data))
df.printSchema()
df.show()

root
 |-- _1: long (nullable = true)
 |-- _2: long (nullable = true)

+---+---+
| _1| _2|
+---+---+
|  0|  0|
|  1|  1|
|  2|  2|
|  3|  3|
|  4|  4|
|  5|  5|
|  6|  6|
|  7|  7|
|  8|  8|
|  9|  9|
+---+---+



In [44]:
# The 'closure' behaviour in RDD doesn't seem to exist for DataFrames

x = 5
df1 = df.filter(df._1 < x)
df1.show()
x = 3
df1.show()

+---+---+
| _1| _2|
+---+---+
|  0|  0|
|  1|  1|
|  2|  2|
|  3|  3|
|  4|  4|
+---+---+

+---+---+
| _1| _2|
+---+---+
|  0|  0|
|  1|  1|
|  2|  2|
|  3|  3|
|  4|  4|
+---+---+



In [45]:
# Because of the Catalyst optimizer !

df1.explain()

== Physical Plan ==
*(1) Filter (isnotnull(_1#1265L) AND (_1#1265L < 5))
+- *(1) Scan ExistingRDD[_1#1265L,_2#1266L]




In [46]:
def f():
    return x/2
x = 5
df1 = df.select(df._1 * 2 + f() + 1 + 1)
df1.explain()
df1.show()

== Physical Plan ==
*(1) Project [(((cast((_1#1265L * 2) as double) + 2.5) + 1.0) + 1.0) AS ((((_1 * 2) + 2.5) + 1) + 1)#1296]
+- *(1) Scan ExistingRDD[_1#1265L,_2#1266L]


+----------------------------+
|((((_1 * 2) + 2.5) + 1) + 1)|
+----------------------------+
|                         4.5|
|                         6.5|
|                         8.5|
|                        10.5|
|                        12.5|
|                        14.5|
|                        16.5|
|                        18.5|
|                        20.5|
|                        22.5|
+----------------------------+



In [48]:
rdd = sc.parallelize(range(10))
x = 5
a = rdd.filter(lambda z: z < x)
print(a.take(10))
x = 3
print(a.take(10))

[0, 1, 2, 3, 4]
[0, 1, 2]


In [50]:
counter = 0

def increment_counter(x):
    global counter
    counter += 1

df.foreach(increment_counter)

print(counter)

0
